In [1]:
# Install required packages
!pip install -U bitsandbytes transformers datasets accelerate seqeval unsloth huggingface_hub torchcrf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━

In [2]:
# Log in so we can pull gated checkpoints
from huggingface_hub import notebook_login
notebook_login()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# Clone FinEntity
!git clone https://github.com/yixuantt/FinEntity.git


Cloning into 'FinEntity'...
remote: Enumerating objects: 118, done.
remote: Total 118 (delta 0), reused 0 (delta 0), pack-reused 118 (from 1)
Receiving objects: 100% (118/118), 28.24 MiB | 12.51 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Error downloading object: model_bert_crf (24132d8): Smudge error: Error downloading model_bert_crf (24132d85e1ad90e91d9688e5ddaf6f520396644386817fa2f877f21a58ea84fe): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

Errors logged to /content/FinEntity/.git/lfs/logs/20250510T214156.227148652.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: model_bert_crf: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'



In [4]:
# Setup imports
import sys, torch, numpy as np, torch.nn as nn
sys.path.append("FinEntity")

from torch.utils.data import Dataset, random_split
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel,
    Trainer, TrainingArguments, DataCollatorForTokenClassification
)
from datasets import load_dataset
from seqeval.metrics import classification_report, f1_score
from sequence_aligner.labelset import LabelSet
from sequence_aligner.containers import TraingingBatch, TrainingExample

# Simple config
class config: dev_split_size = 0.2


In [5]:
!rm -rf ~/.cache/huggingface/hub

In [6]:
# 1) Load + reshape FinEntity
hf = load_dataset("yixuantt/FinEntity", split="train")
raw = [{"text":e["content"],
        "entities":[{"start":a["start"],"end":a["end"],"label":a["tag"]}
                    for a in e["annotations"]]}
       for e in hf]


README.md:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

FinEntity.json:   0%|          | 0.00/710k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/979 [00:00<?, ? examples/s]

In [7]:
# 2) Load gated LLaMA-3 in 4-bit
bnb = BitsAndBytesConfig(load_in_4bit=True, llm_int8_enable_fp32_cpu_offload=True)

# model_name = "unsloth/llama-3-8b-bnb-4bit"
model_name = "unsloth/Qwen2.5-7b-bnb-4bit"

model = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3-8b-bnb-4bit",
    quantization_config = bnb,
    device_map          = "auto",
    use_auth_token      = True,
    trust_remote_code   = True
) # for Llama

# model = AutoModel.from_pretrained(
#     "unsloth/Qwen2.5-7b-bnb-4bit",
#     quantization_config = bnb,
#     device_map          = "auto",
#     token      = True,
#     trust_remote_code   = True
# ) # for Qwen

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast       = True,
    use_auth_token = True
)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(151665, 4096, padding_idx=128255)

In [8]:
# 3) Build token-classification dataset
class TrainingDatasetLLaMA(Dataset):
    def __init__(self, data, tokenizer, tpb=128):
        self.tpb       = tpb
        self.tokenizer = tokenizer
        self.label_map = {"O":0, "Positive":1, "Negative":2, "Neutral":3}
        self.examples  = []
        tokenizer.padding_side = "right"

        for ex in data:
            text, ents = ex["text"], ex["entities"]
            # 1) build char-level labels
            char_lbl = ["O"] * len(text)
            for e in ents:
                for i in range(e["start"], e["end"]):
                    if 0 <= i < len(text):
                        char_lbl[i] = e["label"]

            # 2) tokenize with offsets (no mutation!)
            toks = tokenizer(
                text,
                padding="max_length",
                truncation=True,
                max_length=tpb,
                return_attention_mask=True,
                return_offsets_mapping=True
            )
            input_ids      = toks["input_ids"]
            attention_mask = toks["attention_mask"]
            offsets        = toks["offset_mapping"]

            # 3) align token → label by zipping
            labels = []
            for (start, end), mask_bit in zip(offsets, attention_mask):
                if mask_bit == 0 or start == end:
                    labels.append(0)
                else:
                    span = char_lbl[start:end]
                    # pick any non-O label in the span
                    lbls = set(span) - {"O"}
                    if lbls:
                        labels.append(self.label_map[lbls.pop()])
                    else:
                        labels.append(self.label_map["O"])

            # sanity: all three should be length tpb
            assert len(input_ids) == len(attention_mask) == len(labels) == tpb

            self.examples.append({
                "input_ids":      torch.tensor(input_ids,      dtype=torch.long),
                "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
                "labels":         torch.tensor(labels,         dtype=torch.long),
            })

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]



dataset = TrainingDatasetLLaMA(raw, tokenizer, tpb=128)
n       = len(dataset)
val_sz  = int(config.dev_split_size * n)   # e.g. 10% of total
train_sz = n - val_sz                      # the remaining 90%

train_ds, val_ds = random_split(dataset, [train_sz, val_sz])



In [9]:
ex0 = train_ds[0]
print(ex0["input_ids"].shape, ex0["attention_mask"].shape, ex0["labels"].shape)
print("Unique labels:", set(ex0["labels"].tolist()))

print("Max input ID in dataset:", max([ex["input_ids"].max().item() for ex in train_ds]))
print("Tokenizer vocab size:", tokenizer.vocab_size)

torch.Size([128]) torch.Size([128]) torch.Size([128])
Unique labels: {0, 1}
Max input ID in dataset: 151654
Tokenizer vocab size: 151643


In [10]:
# 4) Define the token-classification head
!pip install pytorch-crf
from torchcrf import CRF

class LlamaForTokenClassificationCRF(nn.Module):
    def __init__(self, base_model, hidden_size, num_labels):
        super().__init__()
        self.base       = base_model
        self.dropout    = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.crf        = CRF(num_tags=num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True
        )
        # Last hidden layer
        sequence_output = self.dropout(outputs.hidden_states[-1])
        emissions = self.classifier(sequence_output)  # (B, T, num_labels)

        if labels is not None:
            # CRF expects mask as ByteTensor or BoolTensor
            loss = -self.crf(emissions, tags=labels, mask=attention_mask.bool(), reduction="mean")
            return loss, emissions
        else:
            # Decode tags
            prediction = self.crf.decode(emissions, mask=attention_mask.bool())
            return prediction

class QwenForTokenClassificationCRF(nn.Module):
    def __init__(self, base_model, hidden_size, num_labels):
        super().__init__()
        self.base = base_model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs[0])
        emissions = self.classifier(sequence_output)

        if labels is not None:
            loss = -self.crf(emissions, labels, mask=attention_mask.bool(), reduction='mean')
            return loss, emissions
        else:
            prediction = self.crf.decode(emissions, mask=attention_mask.bool())
            return prediction

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_tc = LlamaForTokenClassificationCRF(model, model.config.hidden_size, 4).to(device)
# model_tc = QwenForTokenClassificationCRF(model, model.config.hidden_size, 4).to(device)
model_tc.half()


LlamaForTokenClassificationCRF(
  (base): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(151665, 4096, padding_idx=128255)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          (post_

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification
from seqeval.metrics import classification_report, f1_score

# 1) Data collator (pads and uses label_pad_token_id=-100)
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    label_pad_token_id=-100,
)

# 2) Metric helpers
label_inv = {0:"O",1:"Positive",2:"Negative",3:"Neutral"}

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    inputs = trainer._saved_inputs
    input_ids = inputs["input_ids"].cpu().numpy()
    attention_mask = inputs["attention_mask"].cpu().numpy()

    # Step 1: decode emissions using CRF
    emissions = torch.tensor(predictions)  # [batch_size, seq_len, num_labels]
    mask = torch.tensor(attention_mask).bool()
    decoded = model.crf.decode(emissions, mask=mask)  # List[List[int]]

    # Step 2: prepare true and pred labels
    label_map = {0: "O", 1: "Positive", 2: "Negative", 3: "Neutral"}
    true_labels = []
    pred_labels = []

    for label_seq, pred_seq, attn in zip(labels, decoded, attention_mask):
        temp_true, temp_pred = [], []
        for lbl, pred, m in zip(label_seq, pred_seq, attn):
            if m == 1 and lbl != -100:
                temp_true.append(label_map[lbl])
                temp_pred.append(label_map[pred])
        true_labels.append(temp_true)
        pred_labels.append(temp_pred)

    report = classification_report(true_labels, pred_labels, output_dict=True)
    return {
        "eval_f1": f1_score(true_labels, pred_labels),
        "eval_loss": None,
        "f1_positive": report["Positive"]["f1-score"],
        "f1_negative": report["Negative"]["f1-score"],
        "f1_neutral":  report["Neutral"]["f1-score"],
        "micro_avg":   report["micro avg"]["f1-score"],
        "macro_avg":   report["macro avg"]["f1-score"],
        "weighted_avg":report["weighted avg"]["f1-score"],
    }



# 3) TrainingArguments (minimal)
training_args = TrainingArguments(
    output_dir                 = "./llama-tokenclf", # change directory name for Qwen
    num_train_epochs           = 3,
    per_device_train_batch_size= 8,
    per_device_eval_batch_size = 8,
    learning_rate              = 3e-5,
    logging_steps              = 50,
    logging_dir                = "./logs",
    # no evaluation_strategy here
)

# 4) Trainer
class CRFTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        loss, logits = model(**inputs, labels=labels)
        return (loss, (loss, logits)) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        # Save inputs for compute_metrics
        self._saved_inputs = inputs
        return super().prediction_step(model, inputs, prediction_loss_only, ignore_keys)

trainer = CRFTrainer(
    model            = model_tc,
    args             = training_args,
    train_dataset    = train_ds,
    eval_dataset     = val_ds,
    data_collator    = data_collator,
    tokenizer        = tokenizer,
    compute_metrics  = compute_metrics
)

# 5) Train + Evaluate
trainer.train()
metrics = trainer.evaluate()
print("Eval metrics:", metrics)


<ipython-input-11-a2870c5b2f0e>:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CRFTrainer.__init__`. Use `processing_class` instead.
  trainer = CRFTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lw3612 (lw3612-princeton-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,115.260000
100,0.000000
150,0.000000
200,0.000000
250,0.000000


Step,Training Loss
50,115.260000
100,0.000000
150,0.000000
200,0.000000
250,0.000000
